# 1. Импортирование библиотек и модулей

In [35]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import pairwise_distances
import joblib
import pickle
from numpy import savetxt

# 2.Загрузка обучающей и тестовой выборок

In [2]:
fname_tr = "HDAC1/Sets/HDAC1 work set_st.sdf"

mols_tr = []
y_tr = []
for mol in Chem.SDMolSupplier(fname_tr):
    if mol is not None:
        mols_tr.append(mol)
        y_tr.append(mol.GetIntProp("HDAC1"))

In [3]:
fname_ts = "HDAC1/Sets/HDAC1  test  set_st.sdf"

mols_ts = []
y_ts = []
for mol in Chem.SDMolSupplier(fname_ts):
    if mol is not None:
        mols_ts.append(mol)
        y_ts.append(mol.GetIntProp("HDAC1"))

# 3.Расчет дескрипторов обучающей выборки

In [4]:
calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
header = calc.GetDescriptorNames()

In [5]:
descr_tr= []
for m in mols_tr:
    descr_tr.append(calc.CalcDescriptors(m))
x_tr = np.asarray(descr_tr)

In [8]:
savetxt('HDAC1/Models/RDKit/x_tr.csv', x_tr, delimiter=',')

In [9]:
df_RDKit_2D = pd.DataFrame(x_tr,columns=header)

In [10]:
df_RDKit_2D.head(2)

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,12.711630,-0.356398,12.711630,0.02605,0.191369,566.663,528.359,566.296516,220.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0
1,11.099422,-0.342028,11.099422,0.33560,0.200389,439.520,410.288,439.233188,170.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0


In [11]:
x_tr.shape

(169, 208)

# 4. Расчет дескрипторов тестовой выборки

In [12]:
descr_ts = []
for m in mols_ts:
    descr_ts.append(calc.CalcDescriptors(m))
x_ts = np.asarray(descr_ts)

In [13]:
x_ts.shape

(42, 208)

# 5. Построение и валидация модели RF 

## 5.1. Построение модели RF 

In [14]:
seed = 42

In [15]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

In [25]:
param_grid = {"max_features": [x_tr.shape[1] // 10, x_tr.shape[1] // 7, x_tr.shape[1] // 5, x_tr.shape[1] // 3],
              "n_estimators": [100, 250, 500, 1000]}

In [26]:
m = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=2, cv=cv, verbose=1)

In [27]:
m.fit(x_tr, y_tr)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(), n_jobs=2,
             param_grid={'max_features': [20, 29, 41, 69],
                         'n_estimators': [100, 250, 500, 1000]},
             verbose=1)

In [28]:
m.best_params_

{'max_features': 20, 'n_estimators': 100}

In [29]:
best_clf_RF = m.best_estimator_

## 5.2. 5-fold-cross-validation  модели RF

In [32]:
y_pred_CV_RF = cross_val_predict(best_clf_RF, x_tr, y_tr, cv=cv)

In [33]:
confusion_matrix_RF = metrics.confusion_matrix(y_tr, y_pred_CV_RF, labels=[0,1])
Kappa = metrics.cohen_kappa_score(y_tr, y_pred_CV_RF, weights='linear')
TN, FP, FN, TP = confusion_matrix_RF.ravel()
SE = TP/(TP+FN)
SP = TN/(TN+FP)
BA = (SE + SP)/2
print("balanced_accuracy = ", round((BA), 2))
print("SE = ", round((SE), 2))
print("SP = ", round((SP), 2))
print("Kappa = ", round((Kappa), 2))

balanced_accuracy =  0.86
SE =  0.83
SP =  0.9
Kappa =  0.73


In [37]:
pickle.dump(best_clf_RF, open('HDAC1/Models/RDKit/HDAC1_RF_RDKit.pkl', 'wb'))

## 5.3.Y-randomization для модели RF

In [38]:
permutations = 20
score, permutation_scores, pvalue = permutation_test_score(best_clf_RF, x_tr, y_tr,
                                                           cv=cv, scoring='balanced_accuracy',
                                                           n_permutations=permutations,
                                                           n_jobs=-1,
                                                           verbose=1,
                                                           random_state=24)
print('True score = ', score.round(2),
      '\nY-randomization = ', np.mean(permutation_scores).round(2),
      '\np-value = ', pvalue.round(4))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


True score =  0.84 
Y-randomization =  0.5 
p-value =  0.0476


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   28.4s finished


## 5.4. Валидация на внешней тестовой выборке для модели RF

In [39]:
y_pred_rf = best_clf_RF.predict(x_ts)

In [40]:
y_pred_rf

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [41]:
confusion_matrix_ts = metrics.confusion_matrix(y_ts, y_pred_rf, labels=[0,1])

In [42]:
Kappa = metrics.cohen_kappa_score(y_ts, y_pred_rf, weights='linear')
TN, FP, FN, TP = confusion_matrix_ts.ravel()
SE = TP/(TP+FN)
SP = TN/(TN+FP)
BA = (SE + SP)/2
print("balanced_accuracy = ", round((BA), 2))
print("SE = ", round((SE), 2))
print("SP = ", round((SP), 2))
print("Kappa = ", round((Kappa), 2))

balanced_accuracy =  0.83
SE =  0.82
SP =  0.84
Kappa =  0.66


# 6. Расчет AD методом Euclidian distances при K=1 (https://doi.org/10.1021/acs.jcim.0c00415)

In [43]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [44]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,159,160,161,162,163,164,165,166,167,168
0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,1.280280e+08,1.391149e+05,4.289522e+01,1.907878e+01,1.907878e+01,4.235473e+04,6.196773e+01,2.291414e+09,1.800264e+06,5.959334e+06,...,4.865805e+07,1.028784e+07,6.802476e+05,6.629472e+04,1.355859e+06,3.040226e+03,1.364002e+07,3.878144e+01,3.040226e+03,1.781967e+07
2,1.553010e+08,2.287294e+06,7.042727e+01,3.370460e+01,4.228475e+01,4.539443e+04,6.566582e+01,3.924944e+09,4.959703e+06,5.959334e+06,...,5.298948e+07,7.221996e+07,8.616905e+05,7.803856e+04,4.959703e+06,2.804483e+04,1.364002e+07,3.878510e+06,3.108422e+04,1.917553e+07
3,2.415693e+08,3.731344e+06,5.141174e+06,1.129689e+07,1.129689e+07,6.526903e+04,1.261123e+07,4.865780e+09,6.315562e+06,9.837844e+06,...,6.662950e+07,8.582584e+07,1.705746e+06,8.502131e+04,6.759967e+06,3.870699e+04,1.364002e+07,3.878510e+06,4.174635e+04,2.413524e+07
4,2.415693e+08,6.324434e+06,5.141174e+06,2.840235e+07,2.840235e+07,7.343838e+04,3.134539e+07,4.865780e+09,1.265677e+07,9.837844e+06,...,6.662950e+07,8.582584e+07,1.753568e+06,8.502131e+04,1.761647e+07,4.539443e+04,1.878120e+07,5.959334e+06,4.235473e+04,2.593550e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,6.313470e+09,8.860321e+09,8.535829e+09,8.744057e+09,8.744057e+09,8.888361e+09,8.684309e+09,8.888453e+09,8.821607e+09,8.389755e+09,...,8.469200e+09,7.958565e+09,8.878391e+09,8.887612e+09,8.816647e+09,8.888407e+09,8.522189e+09,8.395714e+09,8.888404e+09,8.797472e+09
165,1.635928e+10,1.890613e+10,1.858164e+10,1.878987e+10,1.878987e+10,1.893417e+10,1.873012e+10,1.004581e+10,1.886742e+10,1.843557e+10,...,1.851501e+10,1.800438e+10,1.892420e+10,1.893342e+10,1.886246e+10,1.893422e+10,1.856800e+10,1.844153e+10,1.893422e+10,1.884328e+10
166,2.030497e+10,2.285183e+10,2.252733e+10,2.273556e+10,2.273556e+10,2.287987e+10,2.267581e+10,1.399151e+10,2.281311e+10,2.238126e+10,...,2.246070e+10,2.195007e+10,2.286990e+10,2.287912e+10,2.280815e+10,2.287991e+10,2.251369e+10,2.238722e+10,2.287991e+10,2.278898e+10
167,3.084594e+10,3.339280e+10,3.306830e+10,3.327653e+10,3.327653e+10,3.342084e+10,3.321678e+10,2.453247e+10,3.335408e+10,3.292223e+10,...,3.300167e+10,3.249104e+10,3.341087e+10,3.342009e+10,3.334912e+10,3.342088e+10,3.305466e+10,3.292819e+10,3.342088e+10,3.332995e+10


In [45]:
similarity= neighbors_k

In [46]:
Dmean=np.mean(similarity[1,:])

In [47]:
round(Dmean, 2)

284489713.2

In [48]:
std=np.std(similarity[1,:])

In [49]:
round(std, 2)

2011066570.5

In [50]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

1290022998.45


In [51]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [52]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,2.784121e+06,4.329279e+04,2.031213e+04,2.061381e+08,3.405877e+01,7.820439e+07,5.494529e+04,5.494529e+04,2.087518e+06,4.972222e+04,...,2.544660e+03,3.773912e+06,6.791379e+11,9.994541e+03,5.243951e+06,2.564010e+06,1.701667e+06,1.590873e+06,1.701667e+06,8.022650e+03
1,2.784121e+06,4.710403e+04,3.543677e+04,2.061381e+08,3.805260e+01,1.012505e+08,5.494529e+04,5.494529e+04,2.528758e+06,4.972226e+04,...,6.272950e+04,1.496024e+07,7.025036e+11,1.427133e+04,7.574169e+06,1.525566e+07,1.701667e+06,1.204915e+07,1.701667e+06,5.724655e+04
2,2.784121e+06,8.488359e+04,6.848104e+04,2.924065e+08,7.310267e+01,2.248647e+08,1.038045e+05,1.038045e+05,2.681461e+06,6.008580e+04,...,7.714811e+04,2.757148e+07,7.130446e+11,1.947552e+04,7.574169e+06,1.661152e+07,2.176843e+06,1.204915e+07,2.176843e+06,8.263147e+04
3,9.827112e+06,8.617352e+04,7.256862e+04,3.196794e+08,1.261123e+07,3.073222e+08,4.490987e+05,4.490987e+05,6.456647e+06,2.376706e+05,...,9.791838e+04,2.757148e+07,7.169903e+11,5.446509e+04,3.210807e+07,2.157123e+07,2.176843e+06,1.204915e+07,2.176843e+06,9.960008e+04
4,2.856127e+07,8.617371e+04,9.194950e+04,3.209012e+08,3.134539e+07,3.231661e+08,8.928653e+05,8.928653e+05,7.258511e+06,4.578139e+05,...,9.791838e+04,2.757148e+07,7.270361e+11,1.008273e+05,3.210807e+07,2.337149e+07,8.136176e+06,1.719032e+07,8.136176e+06,1.026397e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,8.687093e+09,8.887205e+09,8.887383e+09,6.761177e+09,8.684309e+09,7.306943e+09,8.881038e+09,8.881038e+09,8.850133e+09,8.884154e+09,...,8.888299e+09,8.711881e+09,7.359245e+11,8.888075e+09,8.296092e+09,8.800036e+09,8.397891e+09,8.523780e+09,8.397891e+09,8.888304e+09
165,1.873291e+10,1.893302e+10,1.893319e+10,1.680699e+10,1.873012e+10,1.735276e+10,1.892685e+10,1.892685e+10,1.889594e+10,1.892997e+10,...,1.893411e+10,1.875769e+10,7.359245e+11,1.893389e+10,1.834190e+10,1.884585e+10,1.844370e+10,1.856959e+10,1.844370e+10,1.893412e+10
166,2.267860e+10,2.287871e+10,2.287889e+10,2.075268e+10,2.267581e+10,2.129845e+10,2.287254e+10,2.287254e+10,2.284164e+10,2.287566e+10,...,2.287980e+10,2.270339e+10,7.359245e+11,2.287958e+10,2.228760e+10,2.279154e+10,2.238940e+10,2.251529e+10,2.238940e+10,2.287981e+10
167,3.321957e+10,3.341968e+10,3.341986e+10,3.129365e+10,3.321678e+10,3.183942e+10,3.341351e+10,3.341351e+10,3.338261e+10,3.341663e+10,...,3.342077e+10,3.324435e+10,7.359245e+11,3.342055e+10,3.282857e+10,3.333251e+10,3.293037e+10,3.305625e+10,3.293037e+10,3.342078e+10


In [53]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[2.78412123e+06 4.32927860e+04 2.03121310e+04 2.06138142e+08
 3.40590000e+01 7.82043888e+07 5.49452860e+04 5.49452890e+04
 2.08751774e+06 4.97222170e+04 2.49301876e+06 1.45547000e+02
 6.63533006e+07 9.54063154e+05 8.91685500e+03 7.91089178e+05
 1.90326000e+02 6.26830000e+01 1.53535123e+06 2.23752560e+04
 8.14930159e+08 9.27660844e+07 8.78978800e+03 4.71705190e+07
 8.79813139e+07 1.46238170e+05 6.00678751e+05 8.79813139e+07
 9.09648800e+03 6.42530000e+03 1.75372161e+07 7.03635692e+05
 2.54466000e+03 3.77391167e+06 6.79137892e+11 9.99454100e+03
 5.24395114e+06 2.56401041e+06 1.70166745e+06 1.59087297e+06
 1.70166745e+06 8.02265000e+03]


In [54]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True False  True
  True  True  True  True  True  True]


In [55]:
print("Coverage = ", sum(cpd_AD) / len(cpd_AD))

Coverage =  0.9761904761904762


In [56]:
print("Индексы соединений, вошедших в AD = ", np.where(cpd_AD != 0)[0])

Индексы соединений, вошедших в AD =  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 35 36 37 38 39 40 41]


# 7. Построение и валидация модели GBM

## 7.1. Построение модели GBM

In [57]:
param_grid = {"n_estimators": [100, 200, 300, 400, 500]}
gbm = GridSearchCV(GradientBoostingClassifier(subsample=0.5, max_features=0.5), 
                   param_grid, n_jobs=2, cv=cv, verbose=1)

In [58]:
gbm.fit(x_tr, y_tr)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=GradientBoostingClassifier(max_features=0.5,
                                                  subsample=0.5),
             n_jobs=2, param_grid={'n_estimators': [100, 200, 300, 400, 500]},
             verbose=1)

In [59]:
gbm.best_params_

{'n_estimators': 500}

In [60]:
best_clf_GBM = gbm.best_estimator_

## 7.2.  5-fold-cross-validation  модели GBM

In [61]:
y_pred_CV_GBM = cross_val_predict(best_clf_GBM, x_tr, y_tr, cv=cv)

In [62]:
confusion_matrix_CV_GBM = metrics.confusion_matrix(y_tr, y_pred_CV_GBM, labels=[0,1])
Kappa = metrics.cohen_kappa_score(y_tr, y_pred_CV_GBM, weights='linear')
TN, FP, FN, TP = confusion_matrix_CV_GBM.ravel()
SE = TP/(TP+FN)
SP = TN/(TN+FP)
BA = (SE + SP)/2
print("balanced_accuracy = ", round((BA), 2))
print("SE = ", round((SE), 2))
print("SP = ", round((SP), 2))
print("Kappa = ", round((Kappa), 2))

balanced_accuracy =  0.87
SE =  0.84
SP =  0.89
Kappa =  0.73


In [63]:
pickle.dump(best_clf_RF, open('HDAC1/Models/RDKit/HDAC1_GBM_RDKit.pkl', 'wb'))

## 7.3.Y-randomization для модели GBM

In [64]:
permutations = 20
score, permutation_scores, pvalue = permutation_test_score(best_clf_GBM, x_tr, y_tr,
                                                           cv=cv, scoring='balanced_accuracy',
                                                           n_permutations=permutations,
                                                           n_jobs=-1,
                                                           verbose=1,
                                                           random_state=24)
print('True score = ', score.round(2),
      '\nY-randomization = ', np.mean(permutation_scores).round(2),
      '\np-value = ', pvalue.round(4))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


True score =  0.86 
Y-randomization =  0.49 
p-value =  0.0476


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.2min finished


## 7.4. Валидация на тестовой выборке для модели GBM

In [65]:
y_pred_gbm = best_clf_GBM.predict(x_ts)

In [66]:
y_pred_gbm

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [67]:
confusion_matrix_GBM = metrics.confusion_matrix(y_ts, y_pred_gbm, labels=[0,1])
Kappa = metrics.cohen_kappa_score(y_ts, y_pred_gbm, weights='linear')
TN, FP, FN, TP = confusion_matrix_GBM.ravel()
SE = TP/(TP+FN)
SP = TN/(TN+FP)
BA = (SE + SP)/2
print("balanced_accuracy = ", round((BA), 2))
print("SE = ", round((SE), 2))
print("SP = ", round((SP), 2))
print("Kappa = ", round((Kappa), 2))

balanced_accuracy =  0.81
SE =  0.82
SP =  0.8
Kappa =  0.61


# 8. Построение и валидация модели SVM

## 8.1. Построение модели SVM

In [91]:
scale = StandardScaler().fit(x_tr)
x_tr_sc = scale.transform(x_tr)

In [92]:
joblib.dump(scale, "HDAC1/Models/RDKit/HDAC1_ws_for SVM.pkl", compress=3)

['HDAC1/Models/RDKit/HDAC1_ws_for SVM.pkl']

In [93]:
param_grid = {"C": [10 ** i for i in range(0, 5)],
              "gamma": [10 ** i for i in range(-6, 0)]}

In [94]:
svm = GridSearchCV(SVC(kernel='rbf', probability=True), param_grid, n_jobs=2, cv=cv, verbose=1)

In [95]:
svm.fit(x_tr_sc, y_tr)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=SVC(probability=True), n_jobs=2,
             param_grid={'C': [1, 10, 100, 1000, 10000],
                         'gamma': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]},
             verbose=1)

In [96]:
best_clf_SVM = svm.best_estimator_

## 8.2. 5-fold-cross-validation модели SVM

In [99]:
y_pred_CV_SVM = cross_val_predict(best_clf_SVM, x_tr_sc, y_tr, cv=cv)

In [100]:
confusion_matrix_CV_SVM = metrics.confusion_matrix(y_tr, y_pred_CV_SVM, labels=[0,1])
Kappa = metrics.cohen_kappa_score(y_tr, y_pred_CV_SVM, weights='linear')
TN, FP, FN, TP = confusion_matrix_CV_SVM.ravel()
SE = TP/(TP+FN)
SP = TN/(TN+FP)
BA = (SE + SP)/2
print("balanced_accuracy = ", round((BA), 2))
print("SE = ", round((SE), 2))
print("SP = ", round((SP), 2))
print("Kappa = ", round((Kappa), 2))

balanced_accuracy =  0.84
SE =  0.79
SP =  0.89
Kappa =  0.68


In [101]:
pickle.dump(best_clf_RF, open('HDAC1/Models/RDKit/HDAC1_SVM_RDKit.pkl', 'wb'))

## 8.3.Y-randomization для модели SVM

In [102]:
permutations = 20
score, permutation_scores, pvalue = permutation_test_score(best_clf_SVM, x_tr_sc, y_tr,
                                                           cv=cv, scoring='balanced_accuracy',
                                                           n_permutations=permutations,
                                                           n_jobs=-1,
                                                           verbose=1,
                                                           random_state=24)
print('True score = ', score.round(2),
      '\nY-randomization = ', np.mean(permutation_scores).round(2),
      '\np-value = ', pvalue.round(4))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


True score =  0.84 
Y-randomization =  0.51 
p-value =  0.0476


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    2.1s finished


## 8.4. Валидация на тестовой выборке для модели SVM

In [103]:
scale = joblib.load("HDAC1/Models/RDKit/HDAC1_ws_for SVM.pkl")
x_ts_sc = scale.transform(x_ts)

In [104]:
y_pred_SVM = best_clf_SVM.predict(x_ts_sc)

In [105]:
y_pred_SVM

array([0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [106]:
confusion_matrix_SVM = metrics.confusion_matrix(y_ts, y_pred_SVM, labels=[0,1])
Kappa = metrics.cohen_kappa_score(y_ts, y_pred_SVM, weights='linear')
TN, FP, FN, TP = confusion_matrix_SVM.ravel()
SE = TP/(TP+FN)
SP = TN/(TN+FP)
BA = (SE + SP)/2
print("balanced_accuracy = ", round((BA), 2))
print("SE = ", round((SE), 2))
print("SP = ", round((SP), 2))
print("Kappa = ", round((Kappa), 2))

balanced_accuracy =  0.76
SE =  0.65
SP =  0.88
Kappa =  0.54


# 9. Консенсусное моделирование

## 9.1.  5-fold CV

In [112]:
y_pred_cv_con = 1 * (((y_pred_CV_RF + y_pred_CV_GBM + y_pred_CV_SVM) / 3) >= 0.5)

In [113]:
confusion_matrix_cv_con = metrics.confusion_matrix(y_tr, y_pred_cv_con, labels=[0,1])
Kappa = metrics.cohen_kappa_score(y_tr, y_pred_cv_con, weights='linear')
TN, FP, FN, TP = confusion_matrix_cv_con.ravel()
SE = TP/(TP+FN)
SP = TN/(TN+FP)
BA = (SE + SP)/2
print("balanced_accuracy = ", round((BA), 2))
print("SE = ", round((SE), 2))
print("SP = ", round((SP), 2))
print("Kappa = ", round((Kappa), 2))

balanced_accuracy =  0.89
SE =  0.84
SP =  0.93
Kappa =  0.78


## 9.2. Test set

In [114]:
pred_c = 1 * (((y_pred_rf + y_pred_gbm + y_pred_SVM) / 3) >= 0.5)

In [115]:
pred_c

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [116]:
confusion_matrix_GBM = metrics.confusion_matrix(y_ts, pred_c, labels=[0,1])
Kappa = metrics.cohen_kappa_score(y_ts, pred_c, weights='linear')
TN, FP, FN, TP = confusion_matrix_GBM.ravel()
SE = TP/(TP+FN)
SP = TN/(TN+FP)
BA = (SE + SP)/2
print("balanced_accuracy = ", round((BA), 2))
print("SE = ", round((SE), 2))
print("SP = ", round((SP), 2))
print("Kappa = ", round((Kappa), 2))

balanced_accuracy =  0.83
SE =  0.82
SP =  0.84
Kappa =  0.66
